# KG Rag

In [ ]:
! pip install --upgrade pandas langchain_neo4j neo4j langchain_core langchain_ollama langchain_groq

**Connection to constructed Neo4j graph**

In [39]:
# 1. Importing Required Libraries
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph

# 2. Connect to Neo4j
NEO4J_URI="neo4j+s://07561b92.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="2wQMIQq_1cANjkh837eMSHUsYBH2F-XLialhvPmM3Bw"

URI = NEO4J_URI
AUTH = (NEO4J_USERNAME,NEO4J_PASSWORD) #AUTH = ("<Username>", "<Password>")
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
print(graph)
print("Connected to Neo4j graph.")

Connected to Neo4j graph.


**Connectivity Check**

In [40]:
graph.query(
"""
MATCH (d:Disease)<-[:HAS_SYMPTOM]-(s:Symptom)
WHERE s.name IN ["itching", "skin_rash", "nodal_skin_eruptions"]
RETURN d.name AS Disease, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC
LIMIT 5

""")

[{'Disease': 'Fungal Infection', 'MatchingSymptoms': 3},
 {'Disease': 'Drug Reaction', 'MatchingSymptoms': 2},
 {'Disease': 'Chickenpox', 'MatchingSymptoms': 2},
 {'Disease': 'Jaundice', 'MatchingSymptoms': 1},
 {'Disease': 'Chronic Cholestasis', 'MatchingSymptoms': 1}]

**LLM connection**

In [41]:
import os
from langchain_groq import ChatGroq

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Initialize the ChatGroq model
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=1,
    max_tokens=1024,
)


**Connectivity Check**

In [42]:
# Define the conversation
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Who is Abdul kalam in 2 lines?"),
]

# Generate the response
ai_msg = llm.invoke(messages)
print(ai_msg.content)


Abdul Kalam was the 11th President of India, serving from 2002 to 2007, and a renowned aerospace engineer and scientist. He is often referred to as the "Missile Man of India" for his contributions to the country's missile defense program and is remembered for his inspiring leadership and vision for India's development.


**Graph RAG: Neo4j Graph + LLM with Prompt for Cypher Query Generation**

In [43]:
from langchain_neo4j import GraphCypherQAChain
from langchain_ollama import ChatOllama
from langchain_core.prompts.prompt import PromptTemplate

# Define the schema and question
schema = """
Nodes:
- Disease: {name: string}
- Symptom: {name: string}
Relationships:
- (Disease)-[:HAS_SYMPTOM]->(Symptom)
"""

question = "What diseases are associated with itching, skin_rash, nodal_skin_eruptions?"

CYPHER_GENERATION_TEMPLATE = """
Task: Generate a Cypher statement to query a graph database.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Examples: 
# What diseases are associated with itching, skin_rash, nodal_skin_eruptions?
MATCH (d:Disease)<-[:HAS_SYMPTOM]-(s:Symptom)
WHERE s.name IN ["itching", "skin_rash", "nodal_skin_eruptions"]
RETURN d.name AS Disease, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC
LIMIT 5

# What treatments are associated with fever and headache?
MATCH (t:Treatment)<-[:TREATS]-(d:Disease)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE s.name IN ["fever", "headache"]
RETURN t.name AS Treatment, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC

# Which doctors are treating diseases related to fatigue and nausea?
MATCH (doc:Doctor)-[:TREATS]->(d:Disease)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE s.name IN ["fatigue", "nausea"]
RETURN doc.name AS Doctor, COUNT(s) AS MatchingSymptoms
ORDER BY MatchingSymptoms DESC
LIMIT 5

The question is:
{question}"""

# # Initialize the LLM
# llm = ChatOllama(model="deepseek-r1:1.5b", temperature=1.5)

# Create the prompt
prompt = PromptTemplate(input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE)

# Set up the GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [44]:
# 11. Invoke the Query Chain and Get the Result
KG_response= chain.invoke({"query": question})

# 12. Print the Result
print(KG_response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease)<-[:HAS_SYMPTOM]-(s:Symptom) 
WHERE s.name IN ["itching", "skin_rash", "nodal_skin_eruptions"] 
RETURN d.name AS Disease, COUNT(s) AS MatchingSymptoms 
ORDER BY MatchingSymptoms DESC 
LIMIT 5
Full Context:
[{'Disease': 'Fungal Infection', 'MatchingSymptoms': 3}, {'Disease': 'Drug Reaction', 'MatchingSymptoms': 2}, {'Disease': 'Chickenpox', 'MatchingSymptoms': 2}, {'Disease': 'Jaundice', 'MatchingSymptoms': 1}, {'Disease': 'Chronic Cholestasis', 'MatchingSymptoms': 1}]

> Finished chain.
{'query': 'What diseases are associated with itching, skin_rash, nodal_skin_eruptions?', 'result': 'Fungal Infection, Drug Reaction, and Chickenpox are associated with itching, skin_rash, nodal_skin_eruptions. Fungal Infection has the most matching symptoms with 3 matches, while Drug Reaction and Chickenpox each have 2 matching symptoms.'}


# Agentic Rag

In [ ]:
! pip install -U duckduckgo-search langchain_tools

**DuckDuckGo Search Tool with LangChain for Disease Recommendation**

In [46]:
# Import necessary modules
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo search tool
ddg_search = DuckDuckGoSearchRun()

# Define the tools the agent can use
tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Search for diseases based on symptoms using DuckDuckGo"
    )
]

# Initialize the agent with Zero-Shot-React-Description agent type
agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Define the symptoms
symptoms = "itching, skin rash, nodal skin eruptions"

# Query the agent to recommend diseases based on the symptoms
Agent_response = agent.invoke(f"Recommend diseases based on the following symptoms: {symptoms}")

print(Agent_response['output'])



> Entering new AgentExecutor chain...
To recommend diseases based on the given symptoms, I should first search for possible diseases that are associated with itching, skin rash, and nodal skin eruptions.

Action: DuckDuckGo Search
Action Input: {'query': 'diseases with itching skin rash nodal skin eruptions', 'verbose': False}
Observation: A rash is any area of irritated or swollen skin. Rashes may be itchy and painful and appear differently on different skin tones. In some cases, a rash may be a sign of a medical emergency. Skin rashes can be caused by a variety of factors, including allergens, chemicals, drugs, viruses, temperature, and even stress. Here are the 10 most common types of rashes, what causes them, and the typical course of treatment. From eczema to allergic reactions to bug bites, here's what common skin rashes look like in photos, and the symptoms that can help you I.D. the condition. It's a rare but dramatic skin problem with a relentless itch and hard lumps all ove

# KG Rag response + Agentic Rag Response

In [47]:
import os
import re
from langchain_groq import ChatGroq

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Initialize the ChatGroq model
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=1,
    max_tokens=1024,
    stream=True
)

# Example responses
kg_response = "Diabetes, Hypertension, Influenza"
web_response = "Hypertension, Influenza, Asthma"

# Define the conversation
messages = [
    ("system", "You are a highly knowledgeable and detail-oriented medical assistant with the ability to critically analyze responses for errors and inconsistencies."),
    ("human", f"I have two sets of responses related to symptoms:\n"
              f"1. Knowledge Graph Response: {KG_response}\n"
              f"2. Web Search Response: {Agent_response}\n"
              f"\n"
              f"Please follow these steps to refine and merge the responses:\n"
              f"1. Review both responses for mistakes, inconsistencies, and redundant diseases. Ensure no disease is listed more than once and prioritize based on severity and commonality.\n"
              f"2. Suggest a healthy diet to manage or alleviate the symptoms. Include specific foods and drinks to consume and avoid.\n"
              f"3. Recommend safe and easy natural techniques or exercises that can complement medical treatment.\n"
              f"4. Specify the type of doctor or specialist to consult and provide a brief reason for each recommendation.\n"
              f"\n"
              f"After reviewing and enhancing the responses, follow these steps:\n"
              f"    1. List all associated diseases, eliminating duplicates and prioritizing based on severity.\n"
              f"    2. Suggest a healthy diet, including specific foods to consume and avoid.\n"
              f"    3. Recommend natural techniques or exercises that are easy and safe.\n"
              f"    4. Specify the type of doctor or specialist to consult with a brief reason.\n"
              f"Ensure the response is clear, concise, and easy to understand."),
]

# # Generate the response
# for chunk in llm.stream(messages):
#     print(chunk.content, end='', flush=True)

# Stream and clean markdown on the fly
for chunk in llm.stream(messages):
    cleaned_chunk = re.sub(r'(\*\*|\*|__|_|\[.*?\]\(.*?\))', '', chunk.content)
    print(cleaned_chunk, end='', flush=True)

c:\Users\admin\OneDrive\Desktop\Symptoms_based_disease_prediction\symp_venv\Lib\site-packages\langchain_groq\chat_models.py:364: UserWarning: WARNING! stream is not default parameter.
                    stream was transferred to model_kwargs.
                    Please confirm that stream is what you intended.
  warnings.warn(


After reviewing the responses, I have refined and merged the information to provide an accurate and comprehensive answer.

Step 1: Review and Refine Responses
Upon examining the responses, I noticed that the Knowledge Graph Response listed Fungal Infection, Drug Reaction, and Chickenpox as associated diseases, while the Web Search Response recommended Prurigo Nodularis (PN). To prioritize and eliminate duplicates, I have compiled a list of unique diseases associated with the symptoms.

Step 2-4: Enhance Responses
To manage or alleviate the symptoms, I suggest the following:

 A healthy diet rich in fruits, vegetables, whole grains, and lean proteins can help manage symptoms. Specific foods to consume include:
	+ Omega-3 fatty acid-rich foods like salmon, sardines, and walnuts to reduce inflammation
	+ Antioxidant-rich foods like berries, leafy greens, and bell peppers to combat oxidative stress
	+ Probiotic-rich foods like yogurt, kefir, and kimchi to support gut health
 Foods and drin